ref.

https://medium.com/@parikshitsaikia1619/unlock-rags-potential-with-distance-metrics-and-rerankers-42df4f171f5a

In [21]:
import weaviate, os, openai
from langchain.embeddings.openai import OpenAIEmbeddings
TAVILY_API_KEY = ""
Openai_API_KEY = ""
GROQ_API_KEY = ""
coher_API_KEY = ""
client = weaviate.Client(
    url="http://192.168.2.186:8080"
)
os.environ["OPENAI_API_KEY"] =  Openai_API_KEY
openai.api_key =os.getenv("OPENAI_API_KEY")
embed_model = "text-embedding-ada-002"
embeddings = OpenAIEmbeddings(model=embed_model)
def get_embedding(text, engine="text-embedding-ada-002") : 
    res = openai.Embedding.create(input=text,engine=engine)['data'][0]['embedding']
    # from openai import OpenAI
    # embedding_client = OpenAI()
    # res= embedding_client.embeddings.create(input = text, model=engine).data[0].embedding
    return res

/usr/local/lib/python3.10/dist-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.6.5.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [22]:
query="LLaMa3의 구조에 대해 설명해줘"
query_vector = get_embedding(query)
response = client.query.get("Test", ["text","title"]).with_hybrid(query, vector=query_vector).with_limit(6).do()

In [11]:
retrieval_res=[r["text"] for r in response["data"]["Get"]["Test"]]

In [16]:
import cohere

co = cohere.Client(coher_API_KEY)

"""
cohere models
https://docs.cohere.com/docs/models
"""
rerank_res = co.rerank(
    model="rerank-multilingual-v3.0",
    query=query, # search query used for docs retrieval
    documents=retrieval_res, #list of retrieved docs
    top_n=4, # selecting top 4 docs
)

reranked_docs = []


for result in rerank_res.results:
    reranked_docs.append(retrieval_res[result.index])

reranked_context = "\n\n------------------\n\n".join(reranked_docs) #combined into a single string

In [20]:
rerank_res.results

[RerankResponseResultsItem(document=None, index=2, relevance_score=0.98365986),
 RerankResponseResultsItem(document=None, index=0, relevance_score=0.9802431),
 RerankResponseResultsItem(document=None, index=1, relevance_score=0.9129032),
 RerankResponseResultsItem(document=None, index=3, relevance_score=0.00069054146)]

In [18]:
print(reranked_context)

Base architecture: LLaMa 3는 standard decoder-only transformer architecture를 기반으로 개발되었다. Tokenizer size: 128K token 수를 가진 vocab을 사용하여 언어를 보다 효과적으로 encoding하여 모델 성능을 향상시킴. -> LLaMa 2보다 4배 큰 tokenizer size Attention method: 8B, 70B 모두 GQA를 사용하여 모델 추론 효율성을 높였다. -> LLaMa 2는 70에만 GQA 적용 context window: 8,192 token -> LLaMa 2는 4,096였다. Tokenizer size가 커지면 encoding을 효과적/효율적으로 할 수 있는 이유: tokenzer size가 4배 커지면 동일한 데이터가 약 4배정도 압축되어 모델에 입력될 수 있다. 예를 들어 입력한 데이터가 LLaMa 2의 tokenizer로 200 token으로 encoding 될 때, LLaMa 3 tokenizer로는 100개가 되지 않는 token 수로 encoding될 수 있기 때문이다. 그리고 tokenizer size가 커진 만큼 모델이 많이 학습하지 않은 한국어 데이터에 대한 tuning을 수행할 때 vocab에 따로 한국어 token을 추가하지 않아도 된다. Training data size LLaMa 3는 15T개의 token으로 학습되었다. (LLaMa 2는 2T로, 2에 비해 3는 약 7배 더 큰 학습데이터를 사용하였다.) 수집된 데이터는 모두 publicly available source에서 수집되었다. code data: LLaMa 2에 사용된 code data 수보다 4배 더 많음 multilingual use case를 위해 전체 training data의 5% 이상을 high-quality non-English(30개 이상) data로 구성

------------------

LLaMa 3 LLaMA 3는 Meta에서 공개한 Open 